In [1]:
import numpy as np
from gtda.time_series import SlidingWindow
from gtda.diagrams import PersistenceLandscape
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
# !pip install spectrum 
# from spectrum.periodogram import speriodogram
import plotly.express as px

In [2]:
# Read and clean CPI data
cpi = pd.read_html('https://www.usinflationcalculator.com/inflation/consumer-price-index-and-annual-percent-changes-from-1913-to-2008/')
cpi = cpi[0]
cpi.columns = cpi.iloc[1,:]
cpi = cpi.iloc[2:,:]
# cpi.to_csv('CPI Cleaned.csv')

cpi = cpi.melt(id_vars = 'Year', var_name = 'Month', value_name = 'CPI')

cpi = cpi[(cpi['Month'] != 'Avg-Avg') & (cpi['Month'] != 'Dec-Dec') & (cpi['Month'] != 'Avg')]

cpi['Date'] = cpi['Year'] + '-' + cpi['Month']
cpi['Date'] = pd.to_datetime(cpi['Date'])
cpi = cpi.set_index('Date').drop(['Year', 'Month'], axis = 1)

cpi = cpi[cpi['CPI'] != 'Avail.Sept.13']
cpi['CPI'] = cpi['CPI'].astype(float)

In [27]:
# Read and clean GDP data
# https://fred.stlouisfed.org/series/GDPC1
gdp = pd.read_csv('GDPC1.csv')
gdp = gdp.rename(columns = {'DATE': 'Date'})
gdp['Date'] = pd.to_datetime(gdp['Date'])
gdp = gdp.set_index('Date')
gdp['GDPC1'] = gdp['GDPC1'].astype(float)
gdp = gdp.rename(columns = {'GDPC1': 'GDP'})

In [28]:
# Read and clean gas data
gas = pd.read_html('https://www.eia.gov/dnav/pet/hist/LeafHandler.ashx?n=pet&s=emm_epm0_pte_nus_dpg&f=m')
gas = gas[4]
gas = gas.melt(id_vars = 'Year', var_name = 'Month', value_name = 'Gas Price')
gas = gas[gas['Year'].notna()]
gas['Year'] = gas['Year'].astype(int).astype(str)
gas['Date'] = gas['Year'] + '-' + gas['Month']
gas['Date'] = pd.to_datetime(gas['Date'])
gas = gas.set_index('Date').drop(['Year', 'Month'], axis = 1)
gas = gas.sort_index()
gas = gas[gas['Gas Price'].notna()]

In [29]:
# Unemployment index
#https://data.bls.gov/pdq/SurveyOutputServlet
unemployment = pd.read_csv('Unemployment Data.csv')
unemployment['Date'] = unemployment['Year'].astype(str) + '/' + unemployment['Period'].apply(lambda x: x.split('M')[1])
unemployment['Date'] = pd.to_datetime(unemployment['Date'], format = "%Y/%m")
unemployment = unemployment.drop(['Year', 'Period'], axis = 1)
unemployment = unemployment.rename(columns = {'Value': 'Unemployement Index'})
unemployment = unemployment.set_index('Date')


In [30]:
#Join CPI to GDP
data = gdp.join(cpi, how = 'inner')
#Join unemployment to data
data = data.join(unemployment, how = 'inner')
# Join gas to data
# data = data.join(gas, how = 'left')

In [31]:
data

,GDP,CPI,Unemployement Index
Date,,,
1948-01-01,2087.442,23.700,3.4
1948-04-01,2121.899,23.800,3.9
1948-07-01,2134.056,24.400,3.6
1948-10-01,2136.440,24.400,3.7
1949-01-01,2107.001,24.000,4.3
...,...,...,...
2021-04-01,19368.310,267.054,6.0
2021-07-01,19478.893,273.003,5.4
2021-10-01,19806.290,276.589,4.6


In [32]:
# Percent change of GDP and Gas
dataPct = data.pct_change()
dataPct = dataPct.dropna()
# dataPct = data[['GDPC1', 'Gas Price', 'CPI']].pct_change()
# Change B/c already percentage)
# dataPct = dataPct.join(data['CPI'].diff())

In [33]:
# figure(figsize = (20,12))
# sns.lineplot(data = dataPct)

fig = px.line(data,
              width=1000, height=700,)
fig.show()

fig = px.line(dataPct,
              width=1000, height=700,)
fig.show()

In [34]:
window_size = 16
stride = 1
df = dataPct
X = df
y = df.index
SW = SlidingWindow(size=window_size, stride=stride)
X_sw, yr = SW.fit_transform_resample(X, y)

In [35]:
win = 100
fig = px.scatter_3d(x = X_sw[win][:,0],
                 y = X_sw[win][:,1],
                z = X_sw[win][:,2],
                labels = {'x': 'GDP',
                          'y': 'CPI',
                          'z': 'Unemployment'},
              width=1000, height=700,)
fig.show()

In [58]:
# Plot the persistence diagram and landscape for a random point cloud sliding window
from gtda.homology import VietorisRipsPersistence
pointcloud = X_sw[100]

vrp = VietorisRipsPersistence()
vrp.fit_transform_plot(pointcloud.reshape(1, *pointcloud.shape))
plt.show()

pl = PersistenceLandscape()
persistencediagram = vrp.fit_transform(pointcloud.reshape(1, *pointcloud.shape))
landscapedata = pl.fit_transform(persistencediagram)
pl.plot(landscapedata, homology_dimensions = [1], plotly_params=None)

In [37]:
# Functions to compute the Lp norms
# Find the range of x values from the persistence diagram:
def Ftseq(diagram):
    births =[]
    deaths =[]
    for pair in diagram:
        if pair[2] == 1:
            births.append(pair[0])
            deaths.append(pair[1])
    return np.linspace(min(births), max(deaths), 100)

# Calculate Lp norm:
def Lpnorm(tseq, landscapevalues, p = 1):
    norms = []
    if p.lower() == 'auc':
        for point in zip(tseq,landscapevalues):
            norms.append(np.trapz(landscapevalues, tseq))
    else: 
        for point in zip(tseq,landscapevalues):
            norms.append(np.linalg.norm(point, p))        
    return sum(norms)

In [38]:
# Calculate the norms for each of the windows in the multivariate sliding window

# Initialize empty list
Norms = np.empty(window_size)
Norms[:] = np.nan
Norms = list(Norms)
vrp = VietorisRipsPersistence()
pl = PersistenceLandscape()
for pointcloud in X_sw:
    persistencediagram = vrp.fit_transform(pointcloud.reshape(1, *pointcloud.shape))
    landscapedata = pl.fit_transform(persistencediagram)
    tseq = Ftseq(persistencediagram[0])
    yvalues = landscapedata[0][1]
    Norms.append(Lpnorm(tseq, yvalues, p = 'auc'))

In [39]:
data

,GDP,CPI,Unemployement Index
Date,,,
1948-01-01,2087.442,23.700,3.4
1948-04-01,2121.899,23.800,3.9
1948-07-01,2134.056,24.400,3.6
1948-10-01,2136.440,24.400,3.7
1949-01-01,2107.001,24.000,4.3
...,...,...,...
2021-04-01,19368.310,267.054,6.0
2021-07-01,19478.893,273.003,5.4
2021-10-01,19806.290,276.589,4.6


In [45]:
data

,GDP,CPI,Unemployement Index
Date,,,
1948-01-01,0.000000,0.000753,3.4
1948-04-01,0.001945,0.001129,3.9
1948-07-01,0.002631,0.003388,3.6
1948-10-01,0.002765,0.003388,3.7
1949-01-01,0.001104,0.001882,4.3
...,...,...,...
2021-04-01,0.975282,0.916964,6.0
2021-07-01,0.981523,0.939362,5.4
2021-10-01,1.000000,0.952863,4.6


In [46]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

dataNorm = data
dataNorm['GDP'] = scaler.fit_transform(np.array(dataNorm['GDP']).reshape(-1, 1)).reshape(-1)
dataNorm['CPI'] = scaler.fit_transform(np.array(dataNorm['CPI']).reshape(-1, 1)).reshape(-1)
dataNorm['Unemployement Index'] = scaler.fit_transform(np.array(dataNorm['Unemployement Index']).reshape(-1, 1)).reshape(-1)
dataNorm['L1 Norm'] = scaler.fit_transform(np.array(Norms).reshape(-1, 1)).reshape(-1)



In [57]:
# figure(figsize= (16,8))
# sns.lineplot(data = dataNorm)
# plt.title('Normalized Data & L1 Norms')

fig = px.line(dataNorm,
              width=1000, height=700,
             title = 'Normalized Data & L1 Norms')
fig.show()